生成答案是原始文本的Multiple Choice的prompt（考虑了多种语言的格式，使用4options）

In [9]:
from datasets import load_dataset
import os
import json

# Load the dataset
dataset = load_dataset("fzkuji/cmexam", trust_remote_code=True)

# Define the save path
save_path = f"./data/llama-factory"  # Change this path to your local directory
os.makedirs(save_path, exist_ok=True)

# Function to save data as JSON with specified columns
def save_as_json(data, filename):
    file_path = os.path.join(save_path, filename)
    
    data_to_save = []
    for item in data:
        # Extract the option texts and generate option letters dynamically
        option_texts = []
        option_letters = []
        for idx, option in enumerate(item['Options']):
            # Handle different data structures for options
            if isinstance(option, dict):
                option_text = option.get('value', '')
            else:
                option_text = str(option)
            option_texts.append(option_text)
            option_letters.append(chr(65 + idx))  # Generate letters 'A', 'B', etc.
        
        # Create a mapping for the current item
        dict_num = {letter: idx for idx, letter in enumerate(option_letters)}
        
        # Process the answer keys, assuming Answer is a single string of concatenated letters (e.g., "ABC")
        answer_keys = item['Answer']
        if isinstance(answer_keys, str):
            # Split the answer string into individual letters (e.g., "ABC" -> ['A', 'B', 'C'])
            answer_keys = list(answer_keys)
        elif isinstance(answer_keys, list):
            pass  # Already a list
        elif answer_keys is None:
            print(f"Warning: Answer is None for item '{item['Question']}', skipping...")
            continue  # Skip items with no answer
        else:
            # Skip if 'Answer' is not a string or list
            print(f"Warning: Unexpected type for 'Answer' in item '{item['Question']}': {type(answer_keys)}")
            continue
        
        # Map the answer keys to the actual option texts, but skip keys that are out of range
        answer_text = []
        for ans in answer_keys:
            if ans in dict_num:
                idx = dict_num[ans]
                answer_text.append(option_texts[idx])
            else:
                print(f"Warning: Answer '{ans}' not found in options for item '{item['Question']}'")
        
        # Construct the input text
        input_text = f"问题：{item['Question']}\n选项：\n" + "\n".join(
            [f"\t{letter}. {text}。" for letter, text in zip(option_letters, option_texts)]
        )
        
        # Add the instruction, input, and output to the data
        if answer_text:
            data_to_save.append({
                "instruction": "假设您是一名医生，请根据患者的症状回答以下选择题。请您输出答案的文本内容（不包含选项序号）。",
                "input": input_text,
                "output": "，".join(answer_text)  # Join multiple answers with commas
            })
        else:
            print(f"Warning: No valid answers for item '{item['Question']}', skipping...")
    
    # Write the modified data to a JSON file
    with open(file_path, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

# Save the modified data for train, validation, and test splits
save_as_json(dataset['train'], 'train.json')
save_as_json(dataset['validation'], 'validation.json')
save_as_json(dataset['test'], 'test.json')


备注，有Warning的原因是有些题目的答案为空，或者选项不完整。这些题目会被跳过，不会被保存到JSON文件中。